# 121 - Disc Game Prize Fund

## Problem Statement

A bag contains one red disc and one blue disc. In a game of chance a player takes a disc at random and its colour is noted. After each turn the disc is returned to the bag, an extra red disc is added, and another disc is taken at random.

The player pays £1 to play and wins if they have taken more blue discs than red discs at the end of the game.

If the game is played for four turns, the probability of a player winning is exactly 11/120, and so the maximum prize fund the banker should allocate for winning in this game would be £10 before they would expect to incur a loss. Note that any payout will be a whole number of pounds and also includes the original £1 paid to play the game, so in the example given the player actually wins £9.

Find the maximum prize fund that should be allocated to a single game in which fifteen turns are played.

## Solution

We can use the Poisson binomial distribution to solve this problem. It is the probability distribution of a sum of independent Bernouilli trials that are not necessarily identically distributed. Here, instead of a constant probability, we have

\begin{equation}
    p_i = \frac{1}{i + 1}, \qquad i = 1, 2, \cdots, 15.
\end{equation}

The probability of winning the game is the probability of having at least 8 successes among the 15 trials. We can compute the probability mass function of the Poisson binomial distribution using the discrete Fourier transform where the probability of $k$ successes among $n$ trials is given by

\begin{equation}
    P(K = k) = \frac{1}{n + 1} \sum_{\alpha = 0}^n C^{-\alpha k} \prod_{x = 1}^n (1 + (C^\alpha - 1)p_x)
\end{equation}

where

\begin{equation}
    C = \exp \left( \frac{2i\pi}{n + 1} \right)
\end{equation}

and $i = \sqrt{-1}$.

Note that there are several ways to compute the PMF of the Poisson binomial distribution and we could have compute the CDF directly. Below we show an alternative way of getting the PMF using the Direct Convolution algorithm.

The probability of winning the game, $p$, is simply

\begin{equation}
    p = \sum_{k = 8}^{15} P(K = k).
\end{equation}

Once we have the $p$, we need to find the maximum prize $X$. When the player wins, he gets $(X - 1)$ and this happens with probability $p$. Otherwise, with probability $(1 - p)$, the player loses 1. For the banker to be happy, the overall payoff to the player must be negative. Therefore, we have the following equation

\begin{equation}
    (X - 1)p - (1 - p) < 0
\end{equation}

which leads to the solution

\begin{equation}
    X < \frac{1}{p}.
\end{equation}

The solution is the smallest $X$ smaller than $\frac{1}{p}$.

In [24]:
import numpy as np

def poisson_binomial_distribution_dft(k, p):
    # Poisson binomial distribution (discrete Fourier transform)
    n = len(p) - 1
    C = np.exp((2 * 1j * np.pi) / (n + 1))
    curr_sum = 0
    for i in range(n + 1):
        curr_prod = 1
        for j in range(1, n + 1):
            curr_prod *= (1 + (C**i - 1) * p[j])
        curr_sum += C**(-i * k) * curr_prod
    return (curr_sum / (n + 1)).real

In [25]:
probabilities = [0] + [1/x for x in range(2, 17)]

p = 0
for k in range(8, 16):
    p += poisson_binomial_distribution_dft(k, probabilities)
p

0.0004406394650193213

In [26]:
def poisson_binomial_pmf(p):
    """ PMF of Poisson binomial distribution using the Direct Convolution algorithm """

    # Initialize the PMF array with the first element set to 1
    pmf = [1]

    for i, p_i in enumerate(p, start=1):
        # Create a new PMF array for the current iteration with size i+1
        next_pmf = [0] * (i + 1)

        # Calculate the first and the last elements in the nextPMF array
        next_pmf[0] = (1 - p_i) * pmf[0]
        next_pmf[i] = p_i * pmf[i - 1]

         # Calculate the intermediate elements in the nextPMF array
        for k in range(1, i):
            next_pmf[k] = p_i * pmf[k - 1] + (1 - p_i) * pmf[k]

        # Update the PMF array to the nextPMF for the next iteration
        pmf = next_pmf
    return pmf

In [29]:
pmf = poisson_binomial_pmf(probabilities)
p = sum(pmf[8:])

p

0.00044063946502123375

In [30]:
1/p

2269.429044336307

Note that the problem could be solved without knowledge of the Poisson binomial distribution. A simple recursive function would give the probability.

In [34]:
def play(r, score, remain, threshold):
    if remain == 0:
        return (score >= threshold)
    p = (1 / (1 + r))
    return p * play(r + 1, score + 1, remain - 1, threshold) + (1 - p) * play(r + 1, score, remain - 1, threshold)

play(1, 0, 15, 8)

0.0004406394650212338